In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from finance.trading.factor.benchmark import *
from finance.trading.factor.proj_grad import *

/home/georges/github/finance-bot/src/finance/trading/factor/proj_grad.py:165: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
  returns
/home/georges/github/finance-bot/src/finance/trading/factor/proj_grad.py:168: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'C', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  - beta[factor][0]


In [3]:
import pandas as pd
from finance.trading.factor.preprocess import *

max_lag = 250
n_factors = 10
path_price = "/home/georges/compet/orycterope/prices.csv"
prices = pd.read_csv(path_price, sep=";")
preprocessor = PreProcess(max_lag)
X_train0, Y_train0 = preprocessor.split_x_y(
    prices, DataType.PRICE
)
path_ret = "/home/georges/compet/data-challenge-qrt/2022/data.csv"
data = pd.read_csv(path_ret, sep=";")
X_train1, Y_train1 = preprocessor.split_x_y(
    data, DataType.RETURN
)

X_train, Y_train = X_train1.copy(deep=True), Y_train1.copy(deep=True)

In [6]:
bench = Benchmark(
        lag=max_lag, n_factors=n_factors, n_iter=1500, eps=1e-6
    )
stiefel, beta = bench.train(X_train.to_numpy(), Y_train.to_numpy())

Benchmark model:   0%|          | 0/1500 [00:00<?, ?it/s]

Iteration 0 with best train metric: 0.02331346933035975
Iteration 1 with best train metric: 0.03319696206337413
Iteration 3 with best train metric: 0.035302048607635694
Iteration 7 with best train metric: 0.03999544914662605
Iteration 67 with best train metric: 0.0435936893692612
Iteration 138 with best train metric: 0.04433262132928289
Iteration 496 with best train metric: 0.04441416345796451
Iteration 675 with best train metric: 0.04578782187329136
Iteration 1361 with best train metric: 0.05094188095529535


In [7]:
proj = ProjGrad(
    lag=max_lag,
    n_factors=n_factors,
    eps=1.0e-6,
    step_stiefel=0.1,
    step_beta=0.05,
    random_state=1234,
    n_iter=10000,
    use_benchmark=False,
)

In [10]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [16]:
print(proj.metric_train(stiefel, beta, X_train, Y_train))
stiefel_bis, beta_bis = proj.train(X_train.to_numpy(), Y_train.to_numpy(), stiefel, beta)

0.05094188095529535


ProjGrad model:   0%|          | 0/10000 [00:00<?, ?it/s]

Iteration: 494 with best train metric: 0.0509684463662212
Iteration: 498 with best train metric: 0.05106057653208777
Iteration: 192 with best train metric: 0.05114563466064007
Iteration: 466 with best train metric: 0.05120870274027366
Iteration: 132 with best train metric: 0.05126493683224948
Iteration: 70 with best train metric: 0.05129162474275833
Iteration: 161 with best train metric: 0.05130867335066276
Iteration: 271 with best train metric: 0.05136999683579173
Iteration: 60 with best train metric: 0.051384578133798056
Iteration: 402 with best train metric: 0.05139800417477611
Iteration: 122 with best train metric: 0.05145472293014482
Iteration: 341 with best train metric: 0.051503196536216225
Iteration: 482 with best train metric: 0.05151066957174729
Iteration: 333 with best train metric: 0.05151984725561083
Iteration: 371 with best train metric: 0.05155813239387104
Iteration: 373 with best train metric: 0.0515767161711937
Iteration: 327 with best train metric: 0.05160127332422297

In [15]:
print(proj.metric_train(stiefel_bis, beta_bis, X_train, Y_train))

0.13071463478068046


# Test 

In [3]:
a = np.array([1,2,3])
b = np.array([-1,1,-2])
np.any(a<=b)

False

In [12]:
np.any(np.arange(0,20).reshape(5,4) < 0)

False

In [2]:
from finance.trading.factor.__model import *
a = np.linspace(0,20,20).reshape(5,4)
a
m = Model(5,4)

/home/georges/github/finance-bot/src/finance/trading/factor/__model.py:38: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  error = np.abs(stiefel.T @ stiefel - np.eye(n_factors))


In [4]:
is_orthonormal(1.e-6, -np.eye(10))


True

In [1]:
from numba import guvectorize, vectorize, int64, float64
import numpy as np

In [2]:
@guvectorize([(float64[:,:], float64[:, :], float64[:, :])], "(n,m),(m,p)->(n,p)")
def f(a,b,res):
    temp = np.dot(a,b)
    # for i in range(temp.shape[0]):
    #     for j in range(temp.shape[1]):
    #         res[i,j] = temp[i,j]
    res[:] = temp[:]


/tmp/ipykernel_28015/3389921772.py:3: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  temp = np.dot(a,b)


In [5]:
a = np.array([[1,1,2],[3,4,5]])
b = np.array([[10],[11],[12]])
a@b

array([[ 45],
       [134]])

In [11]:
f(a,[b]*3)

array([[[ 45.],
        [134.]],

       [[ 45.],
        [134.]],

       [[ 45.],
        [134.]]])

In [ ]:
c = np.array([[0,1,2],[3,4,5]])